In [5]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

In [6]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math

In [20]:
def Day1(data=get_input(1, 2016)[0].split(', ')):
  r_turns = ['N','E','S','W']
  dirs = {
    'N': (0,1),
    'S': (0,-1),
    'E': (1,0),
    'W': (-1,0),
  }
  def cab_dist(coords):
    return abs(coords[0]) + abs(coords[1])
  def r_turn(dir):
    return r_turns[ (r_turns.index(dir) + 1) % len(r_turns) ]
  def l_turn(dir):
    return r_turns[ (r_turns.index(dir) - 1) % len(r_turns) ]

  def part1():
    coords = (0,0)
    dir = 'N'
    for line in data:
      turn_dir = line[0]
      dist = int(line[1:])
      if turn_dir == 'R':
        dir = r_turn(dir)
      elif turn_dir == 'L':
        dir = l_turn(dir)
      else:
        raise "error"
      (dx, dy) = dirs[dir]
      coords = (coords[0] + dist*dx, coords[1] + dist*dy)
    return cab_dist(coords)

  def part2():
    coords = (0,0)
    dir = 'N'
    seen = set()
    for line in data:
      turn_dir = line[0]
      dist = int(line[1:])
      if turn_dir == 'R':
        dir = r_turn(dir)
      elif turn_dir == 'L':
        dir = l_turn(dir)
      else:
        raise "error"
      (dx, dy) = dirs[dir]
      while dist > 0:
        coords = (coords[0] + dx, coords[1] + dy)
        if coords in seen:
          return cab_dist(coords)
        else:
          seen.add(coords)
        dist -= 1

  return part1(),part2()


Day1()

(291, 159)

In [21]:
def Day2(data=get_input(2, 2016)):
  keypad = {
    (0,0): '1',
    (0,1): '2',
    (0,2): '3',
    (1,0): '4',
    (1,1): '5',
    (1,2): '6',
    (2,0): '7',
    (2,1): '8',
    (2,2): '9',
  }
  dirs = {
    'U': (-1,0),
    'D': (1,0),
    'R': (0,1),
    'L': (0,-1),
  }
  def move(coords, dir):
    (x,y) = coords
    (dx,dy) = dirs[dir]
    x += dx
    y += dy
    if x < 0:
      x = 0
    if x > 2:
      x = 2
    if y < 0:
      y = 0
    if y > 2:
      y = 2
    return (x,y)
  keypad2 = {
    (0,2): '1',
    (1,1): '2',
    (1,2): '3',
    (1,3): '4',
    (2,0): '5',
    (2,1): '6',
    (2,2): '7',
    (2,3): '8',
    (2,4): '9',
    (3,1): 'A',
    (3,2): 'B',
    (3,3): 'C',
    (4,2): 'D',
  }
  def move2(coords, dir):
    (x,y) = coords
    (dx,dy) = dirs[dir]
    next_coords = (x + dx, y + dy)
    if next_coords in keypad2:
      return next_coords
    else:
      return coords


  def part1():
    coords = (1,1)
    code = ''
    for line in data:
      for dir in line:
        coords = move(coords, dir)
      code = f"{code}{keypad[coords]}"
    return code


  def part2():
    coords = (2,0)
    code = ''
    for line in data:
      for dir in line:
        coords = move2(coords, dir)
      code = f"{code}{keypad2[coords]}"
    return code

  return part1(),part2()

Day2()

('69642', '8CB23')

In [22]:
import re
def Day3(data=get_input(3,2016)):
  def poss_tri(sides):
    sides = sorted(sides)
    return sides[0] + sides[1] > sides[2]

  def part1():
    count = 0
    for line in data:
      sides = [int(s) for s in re.split(r'\s+', line)]
      if poss_tri(sides):
        count += 1
    return count

  def part2():
    groups = []
    pending_groups = [ [], [], [] ]
    for line in data:
      ints = [int(s) for s in re.split(r'\s+', line)]
      pending_groups[0].append(ints[0])
      pending_groups[1].append(ints[1])
      pending_groups[2].append(ints[2])
      if len(pending_groups[0]) == 3:
        groups.append(pending_groups[0])
        groups.append(pending_groups[1])
        groups.append(pending_groups[2])
        pending_groups = [ [], [] , [] ]
    count = 0
    for sides in groups:
      if poss_tri(sides):
        count += 1
    return count

  return part1(),part2()
Day3()

(1050, 1921)

In [23]:
def Day4(data=get_input(4,2016)):
  alpha = aoc_utils.ALPHABET.lower()
  def sorted_letters(s):
    from collections import Counter
    from functools import cmp_to_key
    c = Counter(s)
    def cmp_fn(a,b):
      if c[a] > c[b]:
        return 1
      elif c[b] > c[a]:
        return -1
      elif c[a] == c[b]:
        return alpha.index(b) - alpha.index(a)
      else:
        raise "Unexpected"
    return sorted(c, key=cmp_to_key(cmp_fn), reverse=True)

  def part1():
    count = 0
    for line in data:
      is_real = True
      s,s1 = line.split('[')
      s1 = s1[:-1]
      id = int(''.join([c for c in s if c in aoc_utils.NUMERIC]))
      s = [c for c in s if c in alpha]
      _sorted = sorted_letters(s)
      for idx,c in enumerate(s1):
        if _sorted[idx] != c:
          is_real = False
      if is_real:
        count += id
    return count
  def part2():
    def shift(s, id):
      out = []
      for c in s:
        if c == '-':
          c = ' '
        else:
          idx = alpha.index(c)
          next_idx = (idx + id) % len(alpha)
          c = alpha[next_idx]
        out.append(c)
      return ''.join(out)

    for line in data:
      s,s1 = line.split('[')
      s1 = s1[:-1]
      id = int(''.join([c for c in s if c in aoc_utils.NUMERIC]))
      s = [c for c in s if c not in aoc_utils.NUMERIC]
      s = shift(s, id)
      if s == "northpole object storage ":
        return id

  return part1(),part2()

Day4()

(173787, 548)

In [24]:
def Day5(data=get_input(5,2016)):
  md5 = aoc_utils.md5
  id = data[0]
  good_idxs = []

  def part1():
    out = ""
    idx = -1 
    while True:
      idx += 1
      code = f"{id}{idx}"
      hash = md5(code)
      if hash.startswith('0'*5):
        out += hash[5]
        good_idxs.append(idx)
        if len(out) == 8:
          return out

  def part2():
    out = [None]*8
    found = 0
    idx = -1
    valid_idx_chars = '01234567'
    while True:
      idx += 1
      if idx < good_idxs[-1] and idx not in good_idxs:
        continue
      code = f"{id}{idx}"
      hash = md5(code)
      if hash.startswith('0'*5) and hash[5] in valid_idx_chars and out[int(hash[5])] is None:
        out[int(hash[5])] = hash[6]
        found += 1
        if found == 8:
          return ''.join(out)
  return part1(),part2()

result = Day5()
assert result == ('f97c354d', '863dde27')
result

KeyboardInterrupt: 

In [ ]:
def Day6(data=get_input(6, 2016)):
  from collections import Counter
  def day1():
    counters = [Counter() for i in range(8)]
    for line in data:
      for idx,c in enumerate(line):
        counters[idx].update(c)
    return ''.join([ctr.most_common()[0][0] for ctr in counters])
  def day2():
    counters = [Counter() for i in range(8)]
    for line in data:
      for idx,c in enumerate(line):
        counters[idx].update(c)
    return ''.join([ctr.most_common()[-1][0] for ctr in counters])

  return day1(),day2()
Day6()

('tzstqsua', 'myregdnr')

In [ ]:
def Day7(data=get_input(7,2016)):
  def is_abba(s):
    return len(s) == 4 and s[0:2] == s[3:1:-1] and s[0] != s[1]
  assert is_abba("abba")
  assert is_abba("acca")
  assert not is_abba("aaaa")

  def has_abba(s):
    for i in range(0, len(s) - 3):
      if is_abba(s[i:i+4]):
        return True
    return False
  assert has_abba("asdflkjsgahdsflkjsdfxyyx")
  assert has_abba("abbasdflkjsgahdsflkjsdfxyyx")

  def is_aba(s):
    return len(s) == 3 and s[0] == s[2] and s[0] != s[1]

  def invert_aba(s):
    assert is_aba(s)
    return ''.join([ s[1], s[0], s[1] ])

  def get_abas(s):
    abas = []
    for i in range(0, len(s) - 2):
      if is_aba(s[i:i+3]):
        abas.append(s[i:i+3])
    return abas

  assert is_aba('aba')
  assert is_aba('cbc')
  assert invert_aba('aba') == 'bab'
  assert get_abas("sdfabab") == ["aba","bab"]

  def part1():
    count = 0
    for line in data:
      valid_parts = []
      invalid_parts = []
      in_brackets = False
      cur_part = ""
      for c in line:
        if c == "[":
          assert not in_brackets
          in_brackets = True
          valid_parts.append(cur_part)
          cur_part = ""
        elif c == "]":
          assert in_brackets
          in_brackets = False
          invalid_parts.append(cur_part)
          cur_part = ""
        else:
          cur_part = cur_part + c
      if len(cur_part) > 0:
        valid_parts.append(cur_part)
      if any([has_abba(s) for s in valid_parts]):
        if not any([has_abba(s) for s in invalid_parts]):
          count += 1
    return count
  def part2(): #data=['aba[bab]xyz','xyx[xyx]xyx','aaa[kek]eke']):
    count = 0
    for line in data:
      valid_parts = []
      invalid_parts = []
      in_brackets = False
      cur_part = ""
      for c in line:
        if c == "[":
          assert not in_brackets
          in_brackets = True
          valid_parts.append(cur_part)
          cur_part = ""
        elif c == "]":
          assert in_brackets
          in_brackets = False
          invalid_parts.append(cur_part)
          cur_part = ""
        else:
          cur_part = cur_part + c
      if len(cur_part) > 0:
        valid_parts.append(cur_part)
      found = False
      for s in valid_parts:
        abas = get_abas(s)
        for aba in abas:
          inv = invert_aba(aba)
          if any([inv in _s for _s in invalid_parts]):
            found = True
      if found:
        count += 1
    return count
  return part1(),part2()
Day7()


(105, 258)

In [ ]:
def Day8(data=get_input(8,2016)):
  OFF = '.'
  ON = '#'
  X = 50
  Y = 6
  def rect(screen, w, h):
    for x in range(w):
      for y in range(h):
        assert (x,y) in screen
        screen[(x,y)] = ON
  def shiftDown(screen, colNum, dist):
    col = [screen[(colNum,y)] for y in range(Y)]
    on_idxs = [idx for idx in range(len(col)) if col[idx] == ON]
    next_on_idxs = [(idx + dist) % len(col) for idx in on_idxs]
    for y in range(Y):
      screen[(colNum,y)] = OFF
    for idx in next_on_idxs:
      screen[(colNum,idx)] = ON
    return screen

  def shiftRight(screen, rowNum, dist):
    row = [screen[(x,rowNum)] for x in range(X)]
    on_idxs = [idx for idx in range(len(row)) if row[idx] == ON]
    next_on_idxs = [(idx + dist) % len(row) for idx in on_idxs]
    for x in range(X):
      screen[(x,rowNum)] = OFF
    for idx in next_on_idxs:
      screen[(idx,rowNum)] = ON
    return screen
  def part1(return_screen=False):
    screen = { (x,y):OFF for x in range(X) for y in range(Y)}
    for line in data:
      if line.startswith("rect"):
        w,h = map(int, re.findall(r"\d+", line))
        rect(screen, w, h)
      elif line.startswith('rotate row'):
        rowNum,dist = map(int, re.findall(f"\d+", line))
        shiftRight(screen, rowNum, dist)
      elif line.startswith('rotate column'):
        colNum,dist = map(int, re.findall(f"\d+", line))
        shiftDown(screen, colNum, dist)
      else:
        raise "Ooops"
    if return_screen:
      return screen
    else:
      return len([v for v in screen.values() if v == ON])
  def part2():
    screen = part1(return_screen=True)
    out = ""
    for row in range(Y):
      rowStr = ""
      for col in range(X):
        rowStr += ON if screen[(col,row)] == ON else OFF
      out += rowStr + "\n"
    #print(out)
    return "RURUCEOEIL"
  return part1(),part2()#,data
result = Day8()
assert result == (121,'RURUCEOEIL')
result

(121, 'RURUCEOEIL')

In [ ]:
import re
def Day9(data=get_input(9,2016)):
  def part1():
    input = ''.join(data)
    input = re.sub(r'\s+', '', input)
    output = ''
    idx = 0
    while idx < len(input):
      c = input[idx]
      if c == '(':
        marker = re.search(r"\(\d+x\d+\)", input[idx:])[0]
        mlen,mcount = aoc_utils.mapints(marker)
        new_cs = input[idx+len(marker) : idx+len(marker)+mlen]
        output += new_cs * mcount
        idx += len(marker) + mlen
      else:
        output += c
        idx += 1
    return len(output)
  def decompress(s):
    if '(' not in s:
      return len(s)
    c = s[0]
    if c != '(':
      return 1 + decompress(s[1:])
    marker = re.search(r"^\(\d+x\d+\)", s)[0]
    size,repeat = map(int, re.findall(r"\d+", marker))
    return repeat * decompress(s[len(marker):len(marker)+size]) + decompress(s[len(marker)+size:])

  def part2():
    input = ''.join(data)
    input = re.sub(r'\s+', '', input)
    return decompress(input)
  return part1(),part2()
result = Day9()
assert result == (150914, 11052855125)
result

(150914, 11052855125)

In [ ]:
from collections import defaultdict
def Day10(data=get_input(10,2016)):
  def give_to_bot(fromBot, toBot, value, bots):
    assert len(bots[toBot]) < 2
    bots[toBot].append(value)
    bots[fromBot] = [v for v in bots[fromBot] if v != value]
  def give_to_out(fromBot, outNum, value, outputs, bots):
    assert outputs[outNum] is None
    outputs[outNum] = value
    bots[fromBot] = [v for v in bots[fromBot] if v != value]
  def lo_val(botNum, bots):
    assert len(bots[botNum]) == 2
    return min(bots[botNum])
  def hi_val(botNum, bots):
    assert len(bots[botNum]) == 2
    return max(bots[botNum])
  def get_lo_hi(botNum, bots):
    return lo_val(botNum, bots),hi_val(botNum, bots)
  def part1(is_part_2=False):
    SEARCH_LO = 17 
    SEARCH_HI = 61
    bots = defaultdict(lambda: [])
    outputs = defaultdict(lambda: None)
    rules = {}
    for line in data:
      if "goes to" in line:
        assert "goes to bot" in line
        val,botNum = aoc_utils.mapints(line)
        bots[botNum].append(val)
        assert len(bots[botNum]) <= 2
    for line in data:
      if "gives low to" in line:
        a,b,c= aoc_utils.mapints(line)
        fromBot = a
        lowKey = f"o{b}" if "low to output" in line else f"b{b}"
        assert "high to" in line
        hiKey = f"o{c}" if "high to output" in line else f"b{c}"
        assert fromBot not in rules
        rules[fromBot] = [lowKey,hiKey]
    i = 0
    while i < 1000:
      i += 1
      curBots = [botNum for botNum in bots if len(bots[botNum]) == 2]
      for fromBot in curBots:
        lowKey,hiKey = rules[fromBot]
        lowVal,hiVal = get_lo_hi(fromBot, bots)
        lowID = aoc_utils.mapints(lowKey)[0]
        hiID = aoc_utils.mapints(hiKey)[0]
        if not is_part_2:
          if lowVal == SEARCH_LO and hiVal == SEARCH_HI:
            return fromBot
        if is_part_2:
          if all([idx in outputs for idx in [0,1,2]]):
            return outputs[0]*outputs[1]*outputs[2]
        if lowKey.startswith("o"):
          give_to_out(fromBot, lowID, lowVal, outputs, bots)
        else:
          give_to_bot(fromBot, lowID, lowVal, bots)
        if hiKey.startswith("o"):
          give_to_out(fromBot, hiID, hiVal, outputs, bots)
        else:
          give_to_bot(fromBot, hiID, hiVal, bots)
  def part2():
    return part1(is_part_2=True)
  return part1(),part2()
assert Day10() == (73, 3965)

In [ ]:
def Day11(data=get_input(11,2016)):
  def part1():
    pass
  def part2():
    pass
  return part1(),part2(),data
Day11()

(None,
 None,
 ['The first floor contains a promethium generator and a promethium-compatible microchip.',
  'The second floor contains a cobalt generator, a curium generator, a ruthenium generator, and a plutonium generator.',
  'The third floor contains a cobalt-compatible microchip, a curium-compatible microchip, a ruthenium-compatible microchip, and a plutonium-compatible microchip.',
  'The fourth floor contains nothing relevant.'])

In [67]:
data = get_input(11,2016)
data
seen = set()

def solve(is_p1=True):
  ELEMENTS_P1 = ["PRO", "COB","CUR","RUT","PLU"]
  ELEMENTS_P2 = ["PRO", "COB","CUR","RUT","PLU","DIL","ELE"]
  ELEMENTS = ELEMENTS_P1 if is_p1 else ELEMENTS_P2
  CHIP2GEN = { f"{el}M": f"{el}G" for el in ELEMENTS}
  CHIP2GEN_P1 = { f"{el}M": f"{el}G" for el in ELEMENTS_P1}
  CHIP2GEN_P2 = { f"{el}M": f"{el}G" for el in ELEMENTS_P2}

  def is_target(state,elements=ELEMENTS):
    el, floors = state
    if el != 3: return False
    assert len(floors) == 4
    for idx,floor in enumerate(floors):
      if idx < 3 and len(floor) > 0: return False
      if idx == 3 and len(floor) != len(elements)*2: return False
    return True

  target_state_p1 = ( #p1
    3,
    (
      frozenset(),
      frozenset(),
      frozenset(),
      frozenset(['PROG','PROM','COBG','COBM','CURG','CURM','RUTG','RUTM','PLUG','PLUM']),
    )
  )
  assert is_target(target_state_p1, ELEMENTS_P1)
  target_state_p2 = ( #p2
    3,
    (
      frozenset(),
      frozenset(),
      frozenset(),
      frozenset(['PROG','PROM','COBG','COBM','CURG','CURM','RUTG','RUTM','PLUG','PLUM','DILG','DILM','ELEG','ELEM']),
    )
  )
  assert is_target(target_state_p2, ELEMENTS_P2)

  def is_valid(state, chipmap=CHIP2GEN):
    el, floors = state
    assert el >= 0 and el <= 3
    for floor in floors:
      for item in floor:
        if item.endswith('M'):
          if chipmap[item] not in floor and any(x.endswith('G') for x in floor):
            return False
    return True

  state_p1 = (
    0, # el
    ( # floors
      frozenset(["PROG","PROM",]), # floor 1
      frozenset(['COBG','CURG','RUTG','PLUG']), # 2
      frozenset(['COBM',"CURM","RUTM","PLUM"]), # 3
      frozenset()
    )
  )
  assert is_valid(state_p1,chipmap=CHIP2GEN_P1)

  state_p2 = ( #p2
    0, # el
    ( # floors
      frozenset(["PROG","PROM","DILG","DILM","ELEG","ELEM"]), # floor 1
      frozenset(['COBG','CURG','RUTG','PLUG']), # 2
      frozenset(['COBM',"CURM","RUTM","PLUM"]), # 3
      frozenset()
    )
  )
  assert is_valid(state_p2, chipmap=CHIP2GEN_P2)

  start_state = state_p1 if is_p1 else state_p2

  def possible_groups(floor):
    from itertools import combinations
    return list(map(list, combinations(floor, 2))) + list(map(list, combinations(floor, 1)))

  def hash_state(state):
    pairs = []
    el, floors = state
    for elem in ELEMENTS:
      chip = elem+"M"
      gen = elem+"G"
      chip_idx = -1
      gen_idx = -1
      for idx,floor in enumerate(floors):
        if chip in floor: chip_idx = idx
        if gen in floor: gen_idx = idx
      assert chip_idx > -1
      assert gen_idx > -1
      pairs.append(f"{chip_idx}-{gen_idx}")
    return str(el) + ':'.join(sorted(pairs))

  import copy

  steps = 0
  stack = []
  seen = set()

  from heapq import heappop, heappush
  heappush(stack, (steps, start_state))

  while len(stack):
    steps, state = heappop(stack)
    hash = hash_state(state)
    if hash in seen: continue
    seen.add(hash)
    if is_target(state):
      return steps
    el,floors = state
    for next_el in set([x for x in [el-1,el+1] if x >= 0 and x <=3]):
      for group in possible_groups(floors[el]):
        group = set(group)
        next_floors = list(copy.copy(floors))
        next_floors[el] -= group
        next_floors[next_el] |= group
        next_floors = tuple(next_floors)
        next_state = (next_el, next_floors)
        if is_valid(next_state):
          heappush(stack, (steps+1, next_state))

solve(is_p1=True), solve(is_p1=False)


(33, 57)

In [ ]:
def Day12(data=get_input(12,2016)):
  def part1(is_part_2=False):
    pc = 0
    regs = {name:0 for name in "a b c d".split(" ")}
    if is_part_2:
      regs["c"] = 1
    while pc < len(data):
      inst = data[pc]
      if inst.startswith('cpy'):
        nums = aoc_utils.getnums(inst)
        val = None
        if nums:
          val = nums[0]
        else:
          from_reg = inst.split(" ")[1]
          val = regs[from_reg]
        to_reg = inst.split(" ")[2]
        regs[to_reg] = val
        pc += 1
      elif inst.startswith('jnz'):
        toks = inst.split(' ')
        src = toks[1]
        val = None
        if src in regs:
          val = regs[src]
        else:
          val = int(src)
        if val == 0:
          pc += 1
        else:
          size = int(toks[2])
          pc += size
      else:
        assert 'dec' in inst or 'inc' in inst
        toks = inst.split(' ')
        reg = toks[1]
        if toks[0] == 'inc':
          regs[reg] += 1
        else:
          regs[reg] -= 1
        pc += 1
    return regs["a"]

  def part2():
    return part1(is_part_2=True)

  return part1(),part2()
assert Day12() == (318083, 9227737)

(318083, 9227737)

In [ ]:
def Day13(data=get_input(13,2016)):
  def is_open(point,fav):
    x,y = point
    poly = x*x + 3*x + 2*x*y + y + y*y
    val = poly + fav
    count = bin(val)[2:].count('1')
    return count % 2 == 0
    
  # A* w/ priority queue
  def part1():
    fav = int(data[0])
    TARGET = (31,39)
    nodes = []
    pushnode = lambda node,cost: heapq.heappush(nodes, (aoc_utils.manhattan_distance(node, TARGET), cost, node) )
    pushnode( (1,1), 0)
    seen = set()
    while nodes:
      _,cost,node = heapq.heappop(nodes)
      if node == TARGET:
        return cost
      seen.add(node)
      for n in aoc_utils.neighbors(node, only_positive=True, only_cardinal=True):
        if n not in seen and is_open(n, fav):
          pushnode(n,cost+1)
  
  # Floodfill w/ lifo queue (deque)
  def part2():
    fav = int(data[0])
    MAX_DIST = 50
    costs = {}
    nodes = deque()
    nodes.append( (0, (1,1)) )
    while nodes:
      cost,node = nodes.popleft()
      if cost > MAX_DIST:
        continue
      costs[node] = cost
      for n in aoc_utils.neighbors(node, only_positive=True, only_cardinal=True):
        if n not in costs and is_open(n,fav):
          nodes.append((cost+1,n))
    return len(costs)
  return part1(),part2()

assert Day13() == (90, 135)

In [ ]:
def Day14(data=get_input(14, 2016)):
  @functools.lru_cache(maxsize=None,typed=None)
  def md5(s,rehash=0):
    cur = aoc_utils.md5(s)
    for _ in range(rehash):
      cur = aoc_utils.md5(cur)
    return cur

  assert md5("abc0", rehash=2016) == "a107ff634856bb300138cac6568c0f24"

  has3_re = re.compile(r"(.)\1\1")
  def has3(s):
    m = has3_re.search(s)
    return m and m[0]

  def is_key(salt, idx):
    k = f"{salt}{idx}"
    hash = md5(k)
    grp = has3(hash)
    if grp:
      max_idx = idx + 1000
      while idx < max_idx:
        idx += 1
        _k = f"{salt}{idx}"
        _hash = md5(_k)
        if grp[0]*5 in _hash:
          return True

  def is_key_stretched(salt, idx):
    k = f"{salt}{idx}"
    hash = md5(k, rehash=2016)
    grp = has3(hash)
    if grp:
      max_idx = idx + 1000
      while idx < max_idx:
        idx += 1
        _k = f"{salt}{idx}"
        _hash = md5(_k, rehash=2016)
        if grp[0]*5 in _hash:
          return True

  def p1():
    salt = data[0]
    keys = []
    MAX_KEYS = 64

    idx = -1
    while len(keys) < MAX_KEYS:
      idx += 1
      if is_key(salt, idx):
        keys.append(idx)
    return keys[-1]

  def p2():
    salt = data[0]
    keys = []
    MAX_KEYS = 64

    idx = -1
    while len(keys) < MAX_KEYS:
      idx += 1
      if is_key_stretched(salt, idx):
        keys.append(idx)
    return keys[-1]

  return p1(),p2()

# Runs in 1m58s
# assert Day14() == (23890, 22696)

In [ ]:
def Day15(data=get_input(15,2016)):
  def get_discs(data):
    discs = []
    for line in data:
      _,size,_,init = aoc_utils.getnums(line)
      discs.append((size,init))
    return discs
  
  def solve(discs):
    found = False
    for t in itertools.count(1):
      found = True
      for idx,(size,init) in enumerate(discs):
        if (t+idx+1+init) % size != 0:
          found = False
          break
      if found:
        return t

  def p1():
    discs = get_discs(data)
    return solve(discs)

  def p2():
    discs = get_discs(data)
    discs.append((11,0))
    return solve(discs)

  return p1(),p2()

assert Day15() == (148737, 2353212)

In [ ]:
def Day15CRT(data=get_input(15,2016)):
  def get_discs(data):
    discs = []
    for line in data:
      _,size,_,init = aoc_utils.getnums(line)
      discs.append((size,init))
    return discs

  def p1():
    discs = get_discs(data)
    a_s = [-disc[1] - idx - 1 for idx,disc in enumerate(discs)]
    n_s = [disc[0] for disc in discs]
    return aoc_utils.chinese_remainder(n_s, a_s)

  def p2():
    discs = get_discs(data)
    discs.append( (11, 0) )
    a_s = [-disc[1] - idx - 1 for idx,disc in enumerate(discs)]
    n_s = [disc[0] for disc in discs]
    return aoc_utils.chinese_remainder(n_s, a_s)

  return p1(),p2()
assert Day15CRT() == (148737, 2353212)


(148737, 2353212)

In [ ]:
# Advent of Code 2016 Day 15 https://adventofcode.com/2016/day/15
def Day15Z3():
  import z3
  def solve(part2=False):
    s = z3.Solver()
    t = z3.Int('t')
    d1Off,d1Size,d1Init = z3.Ints('d1Off d1Size d1Init')
    d2Off,d2Size,d2Init = z3.Ints('d2Off d2Size d2Init')
    d3Off,d3Size,d3Init = z3.Ints('d3Off d3Size d3Init')
    d4Off,d4Size,d4Init = z3.Ints('d4Off d4Size d4Init')
    d5Off,d5Size,d5Init = z3.Ints('d5Off d5Size d5Init')
    d6Off,d6Size,d6Init = z3.Ints('d6Off d6Size d6Init')
    if part2:
      d7Off,d7Size,d7Init = z3.Ints('d7Off d7Size d7Init')
    s.add(d1Off == 1, d1Size == 5, d1Init == 2)
    s.add(d2Off == 2, d2Size == 13, d2Init == 7)
    s.add(d3Off == 3, d3Size == 17, d3Init == 10)
    s.add(d4Off == 4, d4Size == 3, d4Init == 2)
    s.add(d5Off == 5, d5Size == 19, d5Init == 9)
    s.add(d6Off == 6, d6Size == 7, d6Init == 0)
    if part2:
      s.add(d7Off == 7, d7Size == 11, d7Init == 0)
    s.add( (t + d1Off + d1Init) % d1Size == 0 )
    s.add( (t + d2Off + d2Init) % d2Size == 0 )
    s.add( (t + d3Off + d3Init) % d3Size == 0 )
    s.add( (t + d4Off + d4Init) % d4Size == 0 )
    s.add( (t + d5Off + d5Init) % d5Size == 0 )
    s.add( (t + d6Off + d6Init) % d6Size == 0 )
    if part2:
      s.add( (t + d7Off + d7Init) % d7Size == 0 )
    s.add(t > 0)
    s.check()
    model = s.model()
    print(model)
    return model[t]

  return solve(part2=False), # part2 is too slow, take 15 minutes +

# assert Day15Z3() == (148737)

In [ ]:
def Day16(data=get_input(16,2016)):
  def expand(s):
    a = s
    b = reversed(s)
    b = ["0" if c == "1" else "1" for c in b]
    return a + ["0"] + b

  assert expand(["1"]) == ["1","0","0"]
  assert expand([c for c in "11111"]) == [c for c in "11111000000"]

  def checksum(s,init=True):
    if not init and len(s) % 2 == 1:
      return "".join(s)
    result = ["1" if pair[0] == pair[1] else "0" for pair in aoc_utils.chunker(s,2)]
    return checksum(result,init=False)

  assert checksum([c for c in "110010110100"]) == "100"


  def p1(target=272):
    d = [c for c in data[0]]
    while len(d) < target:
      d = expand(d)
    d = d[0:target]
    return checksum(d)
  def p2():
    return p1(target=35651584)

  return p1(),p2()

# Takes 2:45 minutes
# assert Day16() == ('10010110010011110', '01101011101100011')

In [91]:
data = get_input(17, 2016)[0]
# data = 'hijkl'

DOORS = ["U","D","L","R"]
DOOR_MOVES = {
  'U': (0, -1),
  'D': (0, 1),
  'L': (-1, 0),
  'R': (1, 0)
}

cur = (0,0) # x,y
path = ""

def doors(path, prefix=data):
  hash = aoc_utils.md5(prefix + path)[:4]
  return [d for idx,d in enumerate(DOORS) if hash[idx] in "bcdef"]

def next_positions(pos, path):
  x,y = pos
  out = []
  for door in doors(path):
    dx,dy = DOOR_MOVES[door]
    next_pos = (x+dx, y+dy)
    if next_pos[0] >= 0 and next_pos[0] < 4 and next_pos[1] >= 0 and next_pos[1] < 4:
      out.append((next_pos, door))
  return out

def score(state, is_p1=True):
  _,path = state
  if is_p1: return len(path)
  return 1e10 - len(path)


def solve():
  from heapq import heappush, heappop
  stack = []
  path = ""
  state = (cur, path)
  heappush(stack, (len(path), state))

  while len(stack):
    _, state = heappop(stack)
    pos, path = state
    if pos == (3,3):
      return path
    for opt in next_positions(pos, path):
      next_pos, door = opt
      next_path = path + door
      next_state = (next_pos, next_path)
      heappush(stack, (len(next_path), next_state))

def solve2():
  from heapq import heappush, heappop
  stack = []
  path = ""
  state = (cur, path)
  curmax = -1
  heappush(stack, (1e10 - len(path), state))

  while len(stack):
    _, state = heappop(stack)
    pos, path = state
    if pos == (3,3):
      curmax = max(curmax, len(path))
    else:
      for opt in next_positions(pos, path):
        next_pos, door = opt
        next_path = path + door
        next_state = (next_pos, next_path)
        heappush(stack, (1e10 - len(next_path), next_state))
  return curmax

solve(),solve2()

('DRLRDDURDR', 500)

In [108]:
data = get_input(18, 2016)[0]

# data = ".^^.^.^^^^"

SAFE = '.'
TRAP = '^'

from functools import cache

@cache
def get_row(num):
  if num == 0: return data
  prev = get_row(num-1)
  out = ""
  for idx,ch in enumerate(prev):
    l_idx,c_idx,r_idx = idx-1,idx,idx+1
    l = SAFE if l_idx < 0 else prev[l_idx]
    c = prev[c_idx]
    r = SAFE if r_idx >= len(prev) else prev[r_idx]
    is_trap = (l,c,r) in [(TRAP,TRAP,SAFE), (SAFE,TRAP,TRAP), (TRAP, SAFE, SAFE), (SAFE, SAFE, TRAP)]
    out += TRAP if is_trap else SAFE
  return out

p1 = sum(l.count(SAFE) for l in [get_row(i) for i in range(40)])
p2 = sum(l.count(SAFE) for l in [get_row(i) for i in range(400000)])

p1,p2



(1951, 20002936)

In [114]:
data = int(get_input(19, 2016)[0])

from dataclasses import dataclass

@dataclass
class Node():
  v: int
  nxt: "Node"

  def take(self):
    self.nxt = self.nxt.nxt

def LL(num):
  nodes = [Node(i, None) for i in range(1,num+1)]
  for idx,n in enumerate(nodes):
    if idx + 1 < len(nodes):
      n.nxt = nodes[idx+1]
  nodes[-1].nxt = nodes[0]
  return nodes[0]

p1 = -1
n = LL(data)
while True:
  n.take()
  if n == n.nxt:
    p1 = n.v
    break
  n = n.nxt

def solve(n):
  from collections import deque
  l = deque()
  r = deque()
  for i in range(1,n+1):
    if i <= (n+1) // 2:
      l.append(i)
    else:
      r.appendleft(i)
  
  while len(l) and len(r):
    if len(l) > len(r):
      l.pop()
    else:
      r.pop()

    # rotate the start of l onto the end of r
    r.appendleft(l.popleft())
    l.append(r.pop())
  return l.pop()


p2 = solve(data)

p1, p2

(1830117, 1417887)

In [160]:
data = get_input(20, 2016)
testdata = """5-8
0-2
4-7""".split("\n")
maxint = 4294967295

from dataclasses import dataclass

@dataclass
class Span:
  lo: int
  hi: int

  def __repr__(self):
    if self.lo == self.hi: return str(self.lo)
    return f"{self.lo}-{self.hi}"

  def len(self):
    return self.hi + 1 - self.lo

  def overlap(self, lo, hi):
    if lo > self.hi or hi < self.lo: return [self]
    outs = []
    if self.lo <= lo - 1 <= self.hi:
      outs.append(Span(self.lo, lo - 1))
    if self.lo < hi + 1 <= self.hi:
      outs.append(Span(hi + 1, self.hi))
    return outs

class MultiSpan:
  def __init__(self, lo, hi):
    self.spans = [Span(lo, hi)]

  def overlap(self, lo, hi):
    next_spans = []
    for s in self.spans:
      next_spans.extend(s.overlap(lo, hi))
    self.spans = next_spans

  def min(self):
    if len(self.spans): return self.spans[0].lo
    return -1

  def len(self):
    return sum(s.len() for s in self.spans)

  def __repr__(self) -> str:
    return ",".join(f"{s}" for s in self.spans)

ms = MultiSpan(0,maxint)
for l in data:
  lo,hi = map(int, l.split("-"))
  ms.overlap(lo, hi)
  # print(lo, hi, "::", ms)

p1 = ms.min()
p2 = ms.len()

p1,p2



(14975795, 101)